In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import logging
import math,random
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.utils import train,prepare,evaluate,tune
from models.Attention import Attention
from models.Interactors.FIM import FIM_Interactor
from models.Interactors.KNRM import KNRM_Interactor
from models.Encoders.FIM import FIM_Encoder
from models.Encoders.General import Encoder_Wrapper, Pipeline_Encoder
from models.SFI import SFI, SFI_unified
from configs.ManualConfig import hparams

In [2]:
name='sfi'
hparams['k'] = 10
hparams['device'] = 'cuda:1'
# hparams['scale'] = 'large'
# hparams['batch_size'] = 500
# hparams['multiview'] = True

In [3]:
# hparams['validate'] = True
# torch.cuda.set_device(hparams['device'])
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-06-11 04:25:05,614] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'k': 10, 'coarse': None, 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'multiview': False, 'onehot': False, 'val_freq': 1, 'schedule': None}
[2021-06-11 04:25:05,615] INFO (root) preparing dataset...
[2021-06-11 04:25:07,919] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [6]:
hparams['learning_rate'] = 1e-3

In [7]:
encoder = FIM_Encoder(hparams, vocab)
interactor = FIM_Interactor(encoder.level,hparams['k'])

hparams['threshold'] = 0.2

sfi = SFI(hparams, encoder, interactor).to(hparams['device'])

hparams['name'] = '-'.join([i for i in ['sfi', encoder.name, interactor.name, hparams['coarse']] if i])

# sfi.load_state_dict(torch.load('/home/peitian_zhang/Codes/News-Recommendation/data/model_params/sfi-fim-fim-coarse/large_epoch5_step8458_[hs=50,topk=50,attrs=title].model',map_location=hparams['device'])['model'])

# sfi.load_state_dict(torch.load('/home/peitian_zhang/Codes/News-Recommendation/data/model_params/sfi-fim-fim-coarse/large_epoch5_step19562_[hs=50,topk=5,attrs=title].model',map_location=hparams['device'])['model'])

In [11]:
sfi

ModuleAttributeError: 'SFI' object has no attribute 'attr'

In [8]:
sfi(record)

RuntimeError: The expanded size of the tensor (3) must match the existing size (22) at non-singleton dimension 3.  Target sizes: [10, 5, 50, 3, 3, 150].  Tensor sizes: [10, 1, 50, 22, 3, 150]